In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plot_utils as u
from scipy.optimize import curve_fit

In [ ]:
n = 20000000

# Inter-burst time approximation from Tinkhauser et al., 2017 https://doi.org/10.1093/brain/awx252
# They only gave mean and standard deviation of the burst probability, I assumed normal distribution
# and fitted the reciprocal distribution to the data

variable_ld = np.random.normal(0.58, 0.056, n)
inverse_ld = 1 / variable_ld
variable_pd = np.random.normal(0.7, 0.03, n)
inverse_pd = 1 / variable_pd

In [ ]:
def gaussian(x, c, x_mean, sigma):
    return c * np.exp(-(x - x_mean) ** 2 / (2 * sigma ** 2))

def wald(x, mean, scale, ampl):
    a = ampl * np.sqrt(scale / (2 * np.pi * x ** 3))
    return a * np.exp((-scale * (x - mean) ** 2) / (2 * x * mean ** 2))

In [ ]:
plt.figure(figsize=(20, 10))
# _, _, bars_variable = plt.hist(variable_ld, bins=np.arange(0.4, 1.0, 0.001))
histogram, bin_edges, bars_inverse = plt.hist(inverse_ld, bins=np.arange(1, 3, 0.001))

x_hist = np.zeros(len(histogram))
for i in range(len(histogram)):
    x_hist[i] = (bin_edges[i + 1] + bin_edges[i]) / 2

y_hist = histogram

param_optimised_ld, param_covariance_matrix = curve_fit(gaussian, x_hist, y_hist)
param_optimised_wald_ld, param_covariance_matrix_wald = curve_fit(wald, x_hist, y_hist, maxfev=1000)
line_fit = plt.plot(x_hist, gaussian(x_hist, *param_optimised_ld))
line_fit_wald = plt.plot(x_hist, wald(x_hist, *param_optimised_wald_ld))
plt.legend([bars_inverse, line_fit[0], line_fit_wald[0]], ['inverse', 'gaussian fit', 'wald fit'])


In [ ]:
plt.figure(figsize=(20, 10))
# _, _, bars_variable = plt.hist(variable_pd, bins=np.arange(0.4, 1.0, 0.001))
histogram, bin_edges, bars_inverse = plt.hist(inverse_pd, bins=np.arange(1, 2, 0.001))

x_hist = np.zeros(len(histogram))
for i in range(len(histogram)):
    x_hist[i] = (bin_edges[i + 1] + bin_edges[i]) / 2

y_hist = histogram

param_optimised_pd, param_covariance_matrix_pd = curve_fit(gaussian, x_hist, y_hist)
param_optimised_wald_pd, param_covariance_matrix_wald_pd = curve_fit(wald, x_hist, y_hist, maxfev=1000)
line_fit = plt.plot(x_hist, gaussian(x_hist, *param_optimised_pd))
line_fit_wald = plt.plot(x_hist, wald(x_hist, *param_optimised_wald_pd))
plt.legend([bars_inverse, line_fit[0], line_fit_wald[0]], ['inverse', 'gaussian fit', 'wald fit'])

In [ ]:
print('PD params:')
print(f'Wald: {param_optimised_wald_pd}')
print(f'Gauss: {param_optimised_pd}')
print('LD params:')
print(f'Wald: {param_optimised_wald_ld}')
print(f'Gauss: {param_optimised_ld}')

In [ ]:
# CDF approximation of burst length distribution in LD ON ("healthy") and LD OFF ("parkinsonian") conditions from 
# Tinkhauser et al., 2017 https://doi.org/10.1093/brain/awx252

ld_distribution = [
    (0.1, 0.0),
    (0.2, 0.39),
    (0.3, 0.67),
    (0.4, 0.84),
    (0.5, 0.89),
    (0.6, 0.93),
    (0.7, 0.95),
    (0.8, 0.97),
    (0.9, 0.98),
    (1.0, 1.0),
    ]

pd_distribution = [
    (0.1, 0.0),
    (0.2, 0.28),
    (0.3, 0.53),
    (0.4, 0.69),
    (0.5, 0.79),
    (0.6, 0.84),
    (0.7, 0.88),
    (0.8, 0.90),
    (0.9, 0.92),
    (1.0, 0.97),
    (2.0, 0.99),
    (4.0, 1.0),
]

ld_pause_mean = param_optimised_wald_ld[0]
ld_pause_scale = param_optimised_wald_ld[1]

pd_pause_mean = param_optimised_wald_pd[0]
pd_pause_scale = param_optimised_wald_pd[0]

In [ ]:
def inverse_distribution(distribution, value):
    last_y = 0
    last_x = 0
    for x, y in distribution:
        if y >= value and last_y < value:
            return last_x + (value - last_y) * (x - last_x) / (y - last_y)
        last_x = x
        last_y = y

In [ ]:
plt.figure()
ax = plt.gca()
ax.hist([inverse_distribution(ld_distribution, e) for e in np.random.random(100000)], alpha=0.5, bins=np.arange(0, 1.05, 0.1))
ax.hist([inverse_distribution(pd_distribution, e) for e in np.random.random(100000)], alpha=0.5, bins=np.arange(0, 1.05, 0.1))
plt.xlabel('Burst duration (s)')
plt.legend(['LD ON', 'LD OFF'])

In [ ]:
def generate_bursts(tstart, tstop, burst_length_distribution, burst_pause_mean, burst_pause_scale):
    last_burst_time = tstart
    burst_times = [tstart]
    burst_amplitudes = [-1]
    while last_burst_time < tstop:
        pause_stop = last_burst_time + np.random.default_rng().wald(burst_pause_mean, burst_pause_scale)
        burst_stop = pause_stop + inverse_distribution(burst_length_distribution, np.random.random())
        burst_times.extend([pause_stop, burst_stop])
        burst_amplitudes.extend([0, -1])
        last_burst_time = burst_stop
    return burst_times, burst_amplitudes

In [ ]:
tstart = 6
tstop = 30

t, a = generate_bursts(tstart, tstop, pd_distribution, pd_pause_mean, pd_pause_scale)
stt, modulation_signal = u.burst_txt_to_signal(t, a, tstart, tstop, 0.01)

tld, ald = generate_bursts(tstart, tstop, ld_distribution, ld_pause_mean, ld_pause_scale)
sttld, modulation_signalld = u.burst_txt_to_signal(tld, ald, tstart, tstop, 0.01)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(1000 * stt, modulation_signal)
plt.plot(1000 * sttld, 1.1 + modulation_signalld)

In [ ]:
np.savetxt("burst_times_ld_on.txt", [[1000 * e for e in tld]], delimiter=',')
np.savetxt("burst_times_ld_off.txt", [[1000 * e for e in t]], delimiter=',')
np.savetxt("burst_level_ld_on.txt", [ald], delimiter=',', fmt='%d')
np.savetxt("burst_level_ld_off.txt", [a], delimiter=',', fmt='%d')